<a href="https://colab.research.google.com/github/obdulia1/Aludenarticulo1/blob/master/05_Graficaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://solariabiodata.com.mx/images/solaria_banner.png" width="400">

# Biodata Camp: Python
Bienvenidos al material que hemos preparado para el Biodata Camp: Python 🐍

# Módulo de graficación

<img src="http://assets.stickpng.com/images/5848152fcef1014c0b5e4967.png" alt="drawing" height="100"/>

## Instalando y configurando la MV

El primer paso es instalar biopython y plotly en este entorno

In [ ]:
pip install plotly

In [ ]:
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install biopython
except ImportError:
    pass

     |████████████████████████████████| 2.3MB 8.1MB/s 


A coninuación es necesario importar todas las librerías que vamos a necesitar, algunas de ellas

<img src="https://user-images.githubusercontent.com/1217238/65354639-dd928f80-dba4-11e9-833b-bc3e8c6a737d.png" alt="drawing" height="100"/>
<img src="https://www.kindpng.com/picc/m/574-5747046_python-pandas-logo-transparent-hd-png-download.png" alt="drawing" height="100"/>
<img src="http://biopython.org/DIST/docs/tutorial/images/biopython_logo.svg" alt="drawing" height="100"/>
<img src="https://upload.wikimedia.org/wikipedia/commons/3/37/Plotly-logo-01-square.png" alt="drawing" height="100"/>





In [ ]:
import numpy as np
import pandas as pd
import Bio
import plotly as plt

In [ ]:
import os
import sys

from urllib.request import urlretrieve

from Bio import SeqIO, SearchIO, Entrez
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.Blast import NCBIWWW
from Bio.Data import CodonTable

print("Python version:", sys.version_info)
print("Biopython version:", Bio.__version__)

Python version: sys.version_info(major=3, minor=7, micro=10, releaselevel='final', serial=0)
Biopython version: 1.78


## Introducción a plotly

[Plotly](https://plotly.com/) es un paquete multiplataforma (también está disponible para `R`) el cual tiene como objetivo implementar diversos métodos de visualización de datos. 

In [ ]:
import plotly.express as px

Una vez que se ha importado plotly, podemos generar diversos tipos de gráficas:

In [ ]:
fig = px.line(x=["a","b","c"], y=[1,5,4], title="Figura de ejemplo")
print(fig)

Figure({
    'data': [{'hoverlabel': {'namelength': 0},
              'hovertemplate': 'x=%{x}<br>y=%{y}',
              'legendgroup': '',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'mode': 'lines',
              'name': '',
              'showlegend': False,
              'type': 'scatter',
              'x': array(['a', 'b', 'c'], dtype=object),
              'xaxis': 'x',
              'y': array([1, 5, 4]),
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Figura de ejemplo'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'x'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'y'}}}
})


Y procedemos a generar el gráfico

In [ ]:
fig.show()

## Ejercicio con blast

Ahora utilizaremos datos reales para hacer un ejemplo de visualización. Para ello utilizaremos un [archivo desconocido](https://raw.githubusercontent.com/chris-rands/) que se encuentra en cierta locación 

In [ ]:
input_file = "unknown-sequence.fa"

fasta_loc = ("https://raw.githubusercontent.com/chris-rands/"
             "biopython-coronavirus/master/unknown-sequence.fa")

if not os.path.exists(input_file):
    urlretrieve(fasta_loc, input_file)

In [ ]:
for record in SeqIO.parse(input_file, "fasta"):
    print(record.id)

Unknown_sequence


In [ ]:
record = SeqIO.read(input_file, "fasta")
print("Sequence length (bp)", len(record))

Sequence length (bp) 29903


In [ ]:
record.seq

In [ ]:
print("GC content (%)", GC(record.seq))

In [ ]:
%%time
result_handle = NCBIWWW.qblast("blastn", "nt", record.seq)

CPU times: user 104 ms, sys: 48.5 ms, total: 152 ms
Wall time: 14min 1s


In [ ]:
#with open("my_blast.xml", "w") as out_handle:
#  out_handle.write(result_handle.read())
#result_handle.close()
#result_handle = open("my_blast.xml")

In [ ]:
blast_qresult = SearchIO.read(result_handle, "blast-xml")
print(blast_qresult)

¡Resulta que se trata de una secuencia de Coronavirus!

![](https://lh3.googleusercontent.com/proxy/RDHUm2GZRq_Ja_ipursSal4RCPMIQGduQR1-6TVYl7GnTmhFUksUgRPIr-lXvQ3Rih1rlaCm2NKIOaXhejbVF9aRyuj_E6cT7SJliQ2QlzbZq5OByE9OioqpLTtKzBIxgVSmQ45ICd2XWTbjMQ)

In [ ]:
print(blast_qresult.fragments[0])
print("------SB-------")
print(blast_qresult.hits[0])
print("------SB-------")
print(blast_qresult.hsps[0])

In [ ]:
xevalue=[]
for i in range(50):
  xevalue.append(blast_qresult.hsps[i].evalue)
xevalue

In [ ]:
fig=px.histogram(x=xevalue)
fig.show()

In [ ]:
len(blast_qresult.fragments[0].hit.seq)

29903

In [ ]:
xlengths=[]
for i in range(50):
  xlengths.append(len(blast_qresult.fragments[i].hit.seq))
xlengths

In [ ]:
fig=px.histogram(x=xlengths)
fig.show()

In [ ]:
fig=px.scatter(x=xevalue,y=xlengths)
fig.show()

In [ ]:
npxlengths=np.array(xlengths)
npxevalues=np.array(xevalue)
fig=px.scatter(x=npxevalues[np.where(npxevalues<=0.0)],y=npxlengths[np.where(npxlengths>1000)])
fig.show()

In [ ]:

xbits=[]
for i in range(50):
  xbits.append(blast_qresult.hsps[i].bitscore)
xbits


In [ ]:
npxbits=np.array(xbits)
fig=px.scatter_3d(x=npxevalues[np.where(npxevalues<=0.0)],y=npxlengths[np.where(npxlengths>1000)],z=npxbits[np.where(npxbits>1000)])
fig.show()
fig=px.scatter_3d(x=xevalue,y=xlengths,z=xbits)
fig.show()


In [ ]:
import pandas as pd

In [ ]:
blastMetrics=pd.DataFrame(np.hstack((npxbits[:,None],npxevalues[:,None],npxlengths[:,None])),columns=['bitscore','evalue','length'])
blastMetrics

In [ ]:
fig=px.scatter_3d(blastMetrics,x='bitscore',y='evalue',z='length')
fig.show()